## 连接数据库

In [1]:
from neo4j import GraphDatabase

def connectDB():
    uri = "neo4j://8.130.14.250:7687"
    username = "neo4j"
    password = "Litao0098"
    driver = GraphDatabase.driver(uri, auth=(username, password))
    return driver

In [22]:
# 读取并执行Cypher文件

def excuteCypher(cypher_file):
    with open(cypher_file, "r",encoding="UTF-8",errors='ignore') as file:
        cypher_queries = file.read().split(");")[:-1]  # 分割文件内容为单个Cypher查询
        # print(cypher_queries)
    # 执行Cypher查询
    driver=connectDB()
    with driver.session() as session:
        for query in cypher_queries:
            result = session.run(query+r')')
            # for record in result:
            #     print(record)
    driver.close()

In [ ]:
# cypher_file = "app2.cypher"  # Cypher文件的路径
excuteCypher("app0.cypher")
excuteCypher("app1.cypher")

In [ ]:

from flask import Flask, jsonify
from neo4j import GraphDatabase

app = Flask(__name__)

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

# 连接到Neo4j
conn = Neo4jConnection(uri="bolt://8.130.14.250:7474", user="neo4j", pwd="Litao0098")

@app.route('/data', methods=['GET'])
def get_data():
    query_string = "MATCH (n) RETURN n LIMIT 10"  # 示例查询
    result = conn.query(query_string)
    return jsonify([record["n"].items() for record in result])

if __name__ == '__main__':
    app.run(debug=True)


In [1]:
from flask import Flask, jsonify
from neo4j import GraphDatabase

app = Flask(__name__)

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

# 连接到Neo4j
conn = Neo4jConnection(uri="neo4j://8.130.14.250:7687", user="neo4j", pwd="Litao0098")


In [5]:
@app.route('/data', methods=['POST'])
def get_data():
    query_string = "MATCH (n) RETURN n LIMIT 10"  # 示例查询
    result = conn.query(query_string)
    return jsonify([record["n"].items() for record in result])
    
@app.route('/query', methods=['POST'])
def query_graph():
    data = request.json
    time = data.get('time')
    number = data.get('number')
    
    query_string = """
    MATCH (a)-[r]->(b)
    WHERE a.time = $time AND a.number = $number
    RETURN a, r, b
    """
    parameters = {'time': time, 'number': number}
    result = conn.query(query_string, parameters)
    
    output = []
    for record in result:
        output.append({
            'node_a': record['a'].items(),
            'relationship': dict(record['r'].items()),
            'node_b': record['b'].items()
        })

    return jsonify(output)


In [ ]:
from flask import Flask, jsonify
import json
from neo4j import GraphDatabase
# conn = Neo4jConnection(uri="neo4j://8.130.14.250:7687", user="neo4j", pwd="Litao0098")
conn = Neo4jConnection(uri="neo4j://localhost:7687", user="neo4j", pwd="Litao0098")

In [4]:
import py2neo
from py2neo import Graph,Node,Relationship,NodeMatcher
import csv
g=Graph('neo4j://localhost:7687',user='neo4j',password='Litao0098')
with open('e:/One-Personal/OneDrive/0科研/3-数字孪生/6-建模语言/Neo4J/用于数据导入/Progress.csv','r',encoding='gbk') as f:#填入数据表存放路径
    reader=csv.reader(f)
    next(reader)
    for item in reader:
        s_node=Node("Shotcrete", Proj=item[0], ID=item[1], Entity=item[2], Entity_chn=item[3], Time=item[4],Startmilage=item[5], Endmilage=item[6], Length=item[7], Thickness=item[8], Geometry=item[9],Physics=item[10], State=item[11], Rule=item[12], Behavior=item[13])
        g.merge(s_node,"Shotcrete","ID")

IndexError: list index out of range

In [93]:

node=x[0]["n"]
x=node.start_node
print(x)

<Node element_id='4:71789fe3-e317-46ff-81c0-5a04d644a095:0' labels=frozenset({'VirtualEntity'}) properties={'Geometry': ['Tunnel1', 'Tunnel2', 'Geology'], 'Entity': 'Project', 'ConstructionState': '完成施工', 'Description': '分离式特长超深埋隧道，最大埋深1255.01m。一级公路。', 'Organization': ['设计单位:云南省交通规划设计院', '施工单位:云南省建设'], 'Tunnel': ['SYSR->VERE2', 'SYSR->VERE3'], 'ProjectElement': ['SYSR->S1', 'SYSR->S2'], 'ProjectSchedule': '68 month', 'GeographicLocation': '云南省保山市隆阳区老营乡', 'GUID': 'VERE1', 'Media': 'photo:url1', 'Name': 'Laoying Tunnel'}>
